In [11]:
import pandas as pd
import numpy as np
from collections import Counter
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib


In [5]:
data = pd.read_csv('data/stack-overflow-data.csv')

In [6]:
data.head()

,post,tags
0,what is causing this behavior in our c# datet...,c#
1,have dynamic html load as if it was in an ifra...,asp.net
2,how to convert a float value in to min:sec i ...,objective-c
3,.net framework 4 redistributable just wonderi...,.net
4,trying to calculate and print the mean and its...,python


In [12]:
def tokenizeText(data, num_words):
    """
    Bag of words tokenizer
    Args: training features and number of words
    Returns: bag of words matrix
    """
    tokenize = Tokenizer(num_words=num_words)
    tokenize.fit_on_texts(data)
    joblib.dump(tokenize, 'encoders/tokenize.pkl')
    X = tokenize.texts_to_matrix(data)
    return X

def encodeLabels(labels):
    encoder = LabelEncoder()
    encoder.fit(labels)
    y = encoder.transform(labels)
    num_classes = np.max(y) + 1
    print("num classes: {}".format(num_classes))
    joblib.dump(encoder, 'encoders/encoder.pkl')
    return y

In [13]:
X = pd.DataFrame(tokenizeText(data['post'], 500))

In [14]:
sample = pd.concat([X, pd.DataFrame(encodeLabels(data['tags']))], axis=1)

num classes: 20


In [15]:
train, test = train_test_split(sample, test_size=0.15)

In [19]:
print(test.shape)

(6000, 501)


In [20]:
test, holdout = train_test_split(test, test_size=0.1)

In [22]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,0
34274,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
1090,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
8428,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
8714,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17
18517,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8


In [21]:
train.to_csv('data/post_train.csv',header=None,index=False)

test.to_csv('data/post_test.csv',header=None,index=False)

holdout.to_csv('data/post_holdout.csv', header=None,index=False)

## Automate SageMaker deploy with canned estimator

In [4]:
%load_ext autoreload

%autoreload 2

import os
import sagemaker
import boto3, re

from sagemaker import get_execution_role
from processing import myEncoder, myTokenizer
from sageBot import train, saveAndDeploy, serve

sagemaker_session = sagemaker.Session()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
role = get_execution_role()

## Canned DNN Classifier

```python
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

INPUT_TENSOR_NAME = 'inputs'


def estimator_fn(run_config, params):
    feature_columns = [tf.feature_column.numeric_column(INPUT_TENSOR_NAME, shape=[500])]
    return tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                      hidden_units=[128, 256, 20],
                                      n_classes=20,
                                      config=run_config)

def serving_input_fn():
    feature_spec = {INPUT_TENSOR_NAME: tf.FixedLenFeature(dtype=tf.float32, shape=[500])}
    return tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)()

def train_input_fn(training_dir, params):
    """Returns input function that would feed the model during training"""
    return _generate_input_fn(training_dir, 'assignment_train.csv')


def _generate_input_fn(training_dir, training_filename, shuffle=False):
    training_set = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=os.path.join(training_dir, training_filename), target_dtype=np.int, features_dtype=np.float32)
    
    input_fn = tf.estimator.inputs.numpy_input_fn(
        x={INPUT_TENSOR_NAME:  np.array(training_set.data)}, 
        y=np.array(training_set.target),
        num_epochs=100,
        shuffle=shuffle
    )
    return input_fn


```

## SageMaker deploy

```python

import os
import sagemaker
import boto3, re
import tarfile

from sagemaker import get_execution_role
from dnn_classifier import train_input_fn
from dnn_classifier import estimator_fn
from dnn_classifier import serving_input_fn
from sagemaker.tensorflow.model import TensorFlowModel
from operator import itemgetter

sagemaker_session = sagemaker.Session()

role = get_execution_role()


def train(data_directory, num_steps=1000):
    """
    Args: 1. directory name where training data is stored
          2. Number of steps for training; default 1000
          
    Returns: 1. Trained canned estimator
    """
    classifier = estimator_fn(run_config = None, params = None)
    train_func = train_input_fn(data_directory, params = None)
    classifier.train(input_fn = train_func, steps = num_steps)
    # score
    score = classifier.evaluate(input_fn = train_func, steps = 100)
    print("model evaluation: {}".format(score))
    return classifier

def saveAndDeploy(model):
    """
    Args: 1. trained estimator
    
    Returns: 1. deployed model
    """
    exported_model = model.export_savedmodel(export_dir_base = 'export/Servo/', 
                                             serving_input_receiver_fn = serving_input_fn)
    print (exported_model)
    with tarfile.open('model.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)
    print("model exported")
    sagemaker_session = sagemaker.Session()
    inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')
    sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                      role = role,
                                      entry_point = 'dnn_classifier.py')
    print("model saved")
    predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
    return predictor

class serve(object):
    def __init__(self, model, encoder):
        """
        Args: predictor from deployed endpoint
        """
        self.model = model
        self.text_labels = encoder.classes_
    
    def predict(self, features):
        predictions = self.model.predict(features)
        return self.classPredictions(predictions)
    
    def delete(self):
        sagemaker.Session().delete_endpoint(self.model.endpoint)
        
    def classPredictions(self, predictions):
        predictions = predictions['result']['classifications'][0]['classes']
        predictions = sorted(predictions, key=itemgetter('score'), reverse=True)[:3]
        top3 = dict()
        for b in predictions:
            top3[self.text_labels[int(b['label'])]] = '{:.2%}'.format(b['score'])
        return top3


```

In [6]:
classifier = train('data')

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpca0w0_a5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd6df145a90>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpca0w0_a5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd6df145a90>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Finished evaluation at 2018-02-12-16:17:00


INFO:tensorflow:Finished evaluation at 2018-02-12-16:17:00


INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.823281, average_loss = 0.542682, global_step = 1000, loss = 69.4633


INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.823281, average_loss = 0.542682, global_step = 1000, loss = 69.4633


model evaluation: {'accuracy': 0.82328123, 'average_loss': 0.54268211, 'loss': 69.46331, 'global_step': 1000}


In [7]:
predictor = saveAndDeploy(classifier)

INFO:tensorflow:Restoring parameters from /tmp/tmpca0w0_a5/model.ckpt-1000


INFO:tensorflow:Restoring parameters from /tmp/tmpca0w0_a5/model.ckpt-1000


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


...

b'export/Servo/1518452288'
model exported


INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-879895576733


model saved


INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-879895576733
INFO:sagemaker:Creating model with name: sagemaker-tensorflow-py2-cpu-2018-02-12-16-18-10-609
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-py2-cpu-2018-02-12-16-18-10-609


----------------------------------------------------------------------------------------------------------------------------------------------------!

In [8]:
encoder = myEncoder()

In [11]:
hosted = serve(predictor, encoder)

In [13]:
df = pd.read_csv('data/post_holdout.csv')

In [18]:
sample = df.ix[100][:500].values

In [19]:
hosted.predict(sample)

{'.net': '0.44%', 'android': '0.62%', 'java': '98.84%'}

In [24]:
df.ix[100][500]

11.0

In [26]:
encoder.classes_[11]

'java'

In [27]:
hosted.delete()

INFO:sagemaker:Deleting endpoint with name: sagemaker-tensorflow-py2-cpu-2018-02-12-16-18-10-609
